In [ ]:
import numpy as np
from dataclasses import dataclass
from scipy.optimize import least_squares, fmin
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from typing import Dict, List
import time
from numpy.typing import ArrayLike
from toolkit.loading_util import make_path
from toolkit.tire_model.tire_model_pacejka_2010 import *
from toolkit.tire_model.tire_model_fitting import *
from toolkit.tire_model.ttc_loader import *
from toolkit.tire_model.tire_fitting_masks import *
from toolkit.tire_model.tire_model_utils import *
from toolkit.common.constants import *

In [ ]:
# Get the ttc data for the tire you want to fit
combi_runs, cornering, drive_brake, name = load_runs(get_R20_18x6_7_runs())
# create all the boring lists and stuff
params_list: List = []
error_list: List = []

In [ ]:
# If you want to start from the default tir file
# tire_model = readTIR(make_path('./Data/TTCData/TIR_Templates/FSAE_Defaults.tir'))
# or from a existing set of parameters
tire_model = tire_model_from_arr(H_R20_18X6_7)

In [ ]:
arr_len = 100
kappas = np.linspace(-0.3, 0, arr_len)
fz = 900 # N
phit = 0.0#14
vel = 11.1 # m/s
press = 82700 #68900 # Pa
wheel_speed = 11.1 # m/s
alphas = np.zeros(arr_len)
inputs = np.array([np.ones(arr_len) * fz, kappas, alphas, np.zeros(arr_len), np.ones(arr_len) * phit, np.ones(arr_len) * vel, np.ones(arr_len) * press, np.ones(arr_len) * wheel_speed]).T
out = tire_model.fullSteadyState(inputs, use_turnslip=True)
fx = out[:, 0]
fy = out[:, 1]
mz = out[:, 5]

torque_est = fx * 8 * IN_TO_M / 3.5
# plot Fx vs kappa
fig = go.Figure()
fig.add_trace(go.Scatter(x=kappas, y=torque_est, name='Fx'))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), xaxis_title='SR', yaxis_title='Mx (Nm)', legend=dict(orientation="h", yanchor="bottom", y=0.02, xanchor="right", x=1))
fig.show()
# generate the derivative of Fx vs kappa
dfx_dkappa = np.gradient(torque_est, kappas)
# plot the derivative of Fx vs kappa
fig = go.Figure()
fig.add_trace(go.Scatter(x=kappas, y=dfx_dkappa, name='dFx/dSR'))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), xaxis_title='SR', yaxis_title='dMx/dSR (Nm/SR)', legend=dict(orientation="h", yanchor="bottom", y=0.02, xanchor="right", x=1))
fig.show()
